# Note, this *only* works for observing things in the future. Dealing with the pointing corrections in the analysis is not supported here.

# Set your observing times here

In [1]:
from datetime import datetime
tstart = '2017-06-21T00:00:00'
tend = '2017-06-22T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

# Step 1, get the nominal RA/Dec Position for Jupiter at the start

In [3]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../data')

ts = load.timescale()
planets = load('de436.bsp')


test_time = (tend - tstart)*0.5 + tstart
print(test_time.isoformat())

astro_time = Time(test_time)
t = ts.from_astropy(astro_time)
moon, earth, venus = planets['moon'], planets['earth'], planets['venus']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()

print(ra.to(u.deg), dec.to(u.deg))

print(tstart.timetuple().tm_year, tstart.timetuple().tm_yday)
print(tend.timetuple().tm_year, tend.timetuple().tm_yday)


2017-06-21T12:00:00
51.971289477813315 deg 13.604094026059773 deg
2017 172
2017 173


# Step 2: Go run the code that figured out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in ./orbit_engine that use the version that Karl already compiled for the nuops users.

Do:

`./get_latest_TLE.sh`

adjust the

`run_jupiter_test.sh`

script appropriately, then

`./run_jupiter_test.sh`

Check in the resulting jupiter occultation file into the repo, push it here, and them start below.


# Step 3: Parse the resulting occultation file:

In [4]:
file = '../orbit_engine/moon_172_173.occ'
from lunar_planning import *
import lunar_planning
import importlib
importlib.reload(lunar_planning)
orbits = lunar_planning.parse_occ(file)
orbits

,visible,occulted
0,2017-06-21 00:17:07,2017-06-21 01:13:06
1,2017-06-21 01:53:47,2017-06-21 02:49:45
2,2017-06-21 03:30:27,2017-06-21 04:26:25
3,2017-06-21 05:07:06,2017-06-21 06:03:04
4,2017-06-21 06:43:46,2017-06-21 07:39:44
5,2017-06-21 08:20:25,2017-06-21 09:16:23
6,2017-06-21 09:57:05,2017-06-21 10:53:03
7,2017-06-21 11:33:44,2017-06-21 12:29:43
8,2017-06-21 13:10:24,2017-06-21 14:06:22
9,2017-06-21 14:47:03,2017-06-21 15:43:02


# Use SkyField to get the location of the Moon for each orbit:

In [9]:
importlib.reload(lunar_planning)

from datetime import timedelta
buffer = timedelta(minutes=5)



for ind, orbit in orbits.iterrows():
    if ind != 4:
        continue
        
    tstart = orbit['visible'].to_pydatetime() + buffer
    print(tstart, ' to ', tend)
    tend = orbit['occulted'].to_pydatetime() - buffer

    outfile = 'lunar_pointing_'+tstart.isoformat()+'_'+tend.isoformat()
    
    print(outfile)
    
    lunar_planning.get_lunar_radec(tstart, tend,show=True,
                                  parallax_correction=True,
                                  load_path='../data', steps = 5,
                                  outfile=None)
    
    
    
    
    


2017-06-21 06:48:46  to  2017-06-21 07:34:44
lunar_pointing_2017-06-21T06:48:46_2017-06-21T07:34:44
Aim Time            RA                  Dec
Parallax corection (arcsec) 3947.5892027223126
Offset to Venus (deg) 1.0965525563117535
2017-06-21T06:48:46 49.899798951552675  12.75263576100037

Parallax corection (arcsec) 3022.8838284980284
Offset to Venus (deg) 0.8396899523605634
2017-06-21T06:57:57.600000 49.730347826846256  12.8804294282762

Parallax corection (arcsec) 1131.2199241496885
Offset to Venus (deg) 0.3142277567082468
2017-06-21T07:07:09.200000 49.253955303660724  12.992803081132976

Parallax corection (arcsec) 1497.447709729638
Offset to Venus (deg) 0.41595769714712166
2017-06-21T07:16:20.800000 48.67244279951429  13.056125676954204

Parallax corection (arcsec) 3282.380086412748
Offset to Venus (deg) 0.9117722462257634
2017-06-21T07:25:32.400000 48.230037727480784  13.056031101300738

